# Genetic Algorithm
## Completed by:   
* Denys Botuk

As I observed the discrete optimization approach to our problem and the grid of cells could have a large scale, then a good algorithm to get an optimal air defense systems deployment is genetic algorithm due to its heuristic nature, which gives an ability to work with complex problems with huge scale.
As we could have a large number of cells, where we would deploy air defense systems, then a number of possible combinations is too huge.
Instead of going through all possible combinations, which is very time consuming, we can use this algorithm, which is particularly effective for solving such complex discrete optimization problems, where traditional methods might be inefficient. 

In [1]:
import ipynb.fs.full.ProjectAPI as api
import numpy as np
import random
import tqdm

from numpy.random import rand
from numpy.random import randint

As we prepared simulations with different types of systems, we also have its characteristics. So, let's get it.

In [2]:
ppo_names = list(api.ppo_characteristics.keys())

Now, let's take a basic look at the main idea of the algorithm:

*Initial Population:* The process begins with a set of individuals, known as the population. Each individual represents a possible solution to the problem and is typically encoded as a string of characters, often binary.

*Fitness Function:* Each individual in the population is evaluated using a fitness function. This function determines how close an individual is to the optimal solution of the problem.

*Selection:* Individuals are selected for reproduction based on their fitness. Those with higher fitness have a higher chance of being selected. This is akin to the natural selection process where stronger individuals have a better chance of surviving and reproducing.

*Crossover (Recombination):* Pairs of individuals are combined to form new offspring. This process involves swapping parts of their genetic information, which in the context of a GA, could mean swapping parts of their string representations.

*Mutation:* With a small probability, some parts of the offspring's genetic information are randomly altered. This introduces variability into the population and can help prevent the algorithm from getting stuck in local optima.

*Replacement:* The new offspring are then added to the population, and some of the less fit individuals are removed. The population size usually remains constant from one generation to the next.

*Termination:* This process of selection, crossover, mutation, and replacement continues for many generations, or until maximum number of generations is met.

The main problems of this algorithm application to our problem were:
* general genetic algorithm assumes binary combination of chromosome in gen, but as we have $6$ types of the air defense systems, then gen should contains combination of $7$ digits ($6$ types of the air defense system and $0$, which mean that any system is deployed into the cell).
* algorithm has no restrictions for the number of values in the gen, but we have a static number of the air defense systems.

So, all of these issues were resolved, and I managed to modify the algorithm to have an ability to use it for solving our problem.

Now, move on to the implementation:

The following method will be used for unification of the air defense system in the gen combination given its name:

In [3]:
def get_ppo_index(name):
    '''takes an index of the air defense system, based on its name'''
    return ppo_names.index(name) + 1

The following method is used for the parent individual selection from the generated population:

In [4]:
def selection(population, rates, k=3):
    '''selects the "best" individuals based 3 randomly taken ones from the population'''
    # random individual selection from the population
    selection_index = randint(len(population))
    
    for index in randint(0, len(population), k-1):
        
        # take better one
        if rates[index] < rates[selection_index]:
            selection_index = index
            
    return population[selection_index]

Next method is used for crossover process:

Given two parent individuals and cross rate (probability to do crossover), we combine the parts of parents combinations per each system class and (important!) the number of each system in the combination is saved. 

In [5]:
def crossover(parent1, parent2, cross_rate, class_count):
    '''generates children individuals by the recombination of parents gens'''
    children1, children2 = parent1.copy(), parent2.copy()
    
    # determine if do the crossover
    if rand() < cross_rate:
        
        # do the crossover per each system type
        for ppo_class in range(1, class_count + 1):
            
            # Identify the positions of 1s in each parent
            class_parent1 = [i for i in range(len(parent1)) if parent1[i] == ppo_class]
            class_parent2 = [i for i in range(len(parent2)) if parent2[i] == ppo_class]
            
            # Randomly pair the 1s positions from both parents and swap
            np.random.shuffle(class_parent2)
            
            for pos1, pos2 in zip(class_parent1, class_parent2):
                children1[pos1], children2[pos2] = children2[pos2], children1[pos1]
                
    return [children1, children2]

Mutation process is done by the following procedure:

In [6]:
def mutation(combination, mutation_rate):
    '''performs the mutation between values inside one individual'''
    
    for index in range(len(combination)):
        
        # check if do the mutation
        if rand() < mutation_rate:
            
            # determine index of the chromosome to mutate with
            swap_index = randint(0, len(combination) - 1)
            
            # do the mutation (swap)
            combination[index], combination[swap_index] = combination[swap_index], combination[index]

This method generates the random combination of chromosome in a gen, given initial set of the air defense systems:

In [7]:
def generate_combination(combination_length, ppo_set):
    '''generates chromosomes with the count according to the initialized air defense systems set and make a random combination of this'''
    individual = []
    
    for name in ppo_set.keys():
        # add air defense systems of the specific type with specified count
        individual += [get_ppo_index(name)] * ppo_set[name]
    
    # add indexes for empty cells (with no system inside)
    individual += [0] * (combination_length - len(individual))
    
    # make random combination
    np.random.shuffle(individual)
    
    return individual

The final execution of genetic optimization algorithm is behind:

In [8]:
def genetic_algorithm(fitness, init_ppo_set, systems, cells, shape, combination_length, \
                      iteration_num, population_num, cross_rate, mutation_rate):
    '''executes genetic algorithm'''
    # initial population with random combination
    population = np.array([generate_combination(combination_length, init_ppo_set) for _ in range(population_num)])
    
    # initially set the first individual in the population as the best 
    best_individual, best_fitness = population[0], fitness(population[0], systems, cells, shape)
    
    # execute algorithm 'iteration_num' times
    for gen in tqdm.tqdm(range(iteration_num)):
        
        # calculate fitness values for all candidates in the population
        scores = [fitness(c, systems, cells, shape) for c in population]
        
        for i in range(population_num):
            
            # check for new best solution
            if scores[i] < best_fitness:
                best_individual, best_fitness = population[i], scores[i]
                
            # select parent individuals
            selected = [selection(population, scores) for _ in range(population_num)]
            
            # create children generation
            children = list()
            
            for i in range(0, population_num, 2):
                
                # get selected parents in pairs
                parent1, parent2 = selected[i], selected[i+1]
                
                # crossover and mutation
                for child in crossover(parent1, parent2, cross_rate, len(init_ppo_set)):
                    
                    # mutation
                    mutation(child, mutation_rate)
                    
                    # store the child
                    children.append(child)
                    
            # replace population with children
            population = children
            
    return [best_individual, best_fitness]